<a href="https://colab.research.google.com/github/hasibzunair/neuralwhat/blob/master/colabs/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.


In [0]:
def accuracy(x_test):
  y_pred = model.predict(x_test)
  # get labels from predictions
  pred_labels = np.array([np.argmax(pred) for pred in y_pred])
  #convert ground truths to column values
  y_test = np.argmax(y_test, axis=1)
  accuracy = np.mean((y_test==pred_labels)))
  return accuracy

def evaluate_error(x_test):
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]  
    return error
  

In [5]:
# save files in drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!rm -f inference.zip

In [0]:
!cp "/content/gdrive/My Drive/RBC Localization/works/inference.zip" test

In [10]:
!ls

gdrive	sample_data  test


In [11]:
!unzip test/inference.zip

Archive:  test/inference.zip
  inflating: inference/naive.h5      
  inflating: inference/vgg.h5        
  inflating: inference/x_test.npy    
  inflating: inference/y_test.npy    


In [12]:
!ls

gdrive	inference  sample_data	test


In [13]:
os.listdir("inference")

['x_test.npy', 'naive.h5', 'vgg.h5', 'y_test.npy']

In [14]:
x_test = np.load("inference/x_test.npy")
y_test = np.load("inference/y_test.npy")

x_test.shape, y_test.shape

((2756, 200, 200, 3), (2756, 2))

In [0]:
#plt.imshow(x_test[9])

In [16]:
model_1 = load_model("inference/vgg.h5")
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [17]:
model_2 = load_model("inference/naive.h5")
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 128)       73856     
__________

In [18]:
score_1 = model_1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score_1[0])
print('Test accuracy:', score_1[1])

2756/2756 [==============================] - 1198s 435ms/step
Test loss: 0.13639475404447438
Test accuracy: 0.9499274310595065


In [19]:
score_2 = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score_2[0])
print('Test accuracy:', score_2[1])

2756/2756 [==============================] - 286s 104ms/step
Test loss: 0.25343694605488476
Test accuracy: 0.9499274310595065


In [20]:
# prediction for model 1
y_pred_1 = model_1.predict(x_test, verbose=1)
y_pred_1.shape

2756/2756 [==============================] - 1195s 434ms/step


(2756, 2)

In [21]:
# prediction for model 2
y_pred_2 = model_2.predict(x_test, verbose=1)
y_pred_2.shape

2756/2756 [==============================] - 292s 106ms/step


(2756, 2)

In [22]:
y_pred_1.shape, y_pred_2.shape

((2756, 2), (2756, 2))

In [23]:
# ensemble lol
y_pred = .5 *y_pred_1 + .5 *y_pred_2
y_pred.shape

(2756, 2)

In [26]:
y_pred.shape, y_test.shape

((2756, 2), (2756, 2))

In [27]:
# get labels from predictions
pred_labels = np.array([np.argmax(pred) for pred in y_pred])
pred_labels.shape

(2756,)

In [28]:
#convert ground truths to column values
y_test = np.argmax(y_test, axis=1)
y_test.shape

(2756,)

In [29]:
y_test.shape, pred_labels.shape

((2756,), (2756,))

In [30]:
print('Accuracy by ensembling two models: ',np.mean((y_test==pred_labels)))

Accuracy by ensembling two models:  0.9582728592162555


In [0]:
# .6 .4 -->>> VGG FTW
# Accuracy by ensembling two models:  0.9589985486211902

# .5 .5
# Accuracy by ensembling two models:  0.9582728592162555

In [34]:
x = 0.9582728592162555
y = 0.9499274310595065

(x-y) * 100

0.8345428156748991